In [ ]:
#@title Importing libraries...
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from plotly.offline import plot
#import plotly.graph_objs as go
#import plotly

%matplotlib inline

import numpy as np

import pickle as pk

from torch.utils.data import DataLoader

from torchvision.transforms import transforms
from torchvision.datasets import MNIST
#from torchvision.datasets import MNIST

from IPython import display

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [ ]:
#@title Loading dataset
trainset = MNIST(root='./dataMNIST/',
                 train=True,
                 transform=transforms.ToTensor(),
                 download=True)
testset = MNIST(root='./dataMNIST/',
                train=False,
                transform=transforms.ToTensor(),
                download=True)
labels = ['#0','#1','#2','#3','#4','#5','#6','#7','#8','#9']

N_train = len(trainset)
N_test = len(testset)

batch=128

train_loader = DataLoader(dataset=trainset,
                          batch_size=batch,
                          shuffle=True)

test_loader = DataLoader(dataset=testset,
                         batch_size=batch,
                         shuffle=False)

In [ ]:
#@title Establishing architecture
class AE(nn.Module):
  def __init__(self):
    super(AE,self).__init__()
    
    self.act   = nn.LeakyReLU()
    self.tanh  = nn.Tanh()
    
    self.z_dim = 32
    
    self.enc1  = nn.Conv2d(1,128,5,padding=2,stride=2)
    self.enc1s = nn.Sequential(nn.BatchNorm2d(128),\
                               self.act)
    
    self.enc2  = nn.Conv2d(128,256,5,padding=2,stride=2)
    self.enc2s = nn.Sequential(nn.BatchNorm2d(256),\
                               self.act)
    
    self.enc3  = nn.Linear(7*7*256,1024)
    self.enc3s = nn.Sequential(nn.BatchNorm1d(1024),\
                               self.act)
    
    self.enc4  = nn.Linear(1024,128)
    self.enc4s = nn.Sequential(nn.BatchNorm1d(128),\
                               self.act)
    
    self.logvar = nn.Linear(128,self.z_dim)
    
    self.mu    = nn.Linear(128, self.z_dim)
    
    self.dec1  = nn.Linear(self.z_dim,128)
    self.dec1s = nn.Sequential(nn.BatchNorm1d(128),\
                               self.act)
    
    self.dec2  = nn.Linear(128,1024)
    self.dec2s = nn.Sequential(nn.BatchNorm1d(1024),\
                               self.act)
    
    self.dec3  = nn.Linear(1024,7*7*256)
    self.dec3s = nn.Sequential(nn.BatchNorm1d(7*7*256),\
                               self.act)
    
    self.dec4  = nn.ConvTranspose2d(256,128,5,stride=2,padding=2)
    self.dec4s = nn.Sequential(nn.BatchNorm2d(128),\
                               self.act)
    
    self.dec5  = nn.ConvTranspose2d(128,1,5,stride=2,padding=2)
    self.dec5s = nn.Sequential(nn.BatchNorm2d(1),\
                               self.act)
    
    self.dec6  = nn.ConvTranspose2d(1,1,4,stride=1,padding=0)
    self.dec6s = nn.Sequential(nn.BatchNorm2d(1),\
                               nn.Sigmoid())
    
    #nn.init.xavier_normal_(self.enc1.weight.data)
    #nn.init.xavier_normal_(self.enc2.weight.data)
    #nn.init.xavier_normal_(self.enc3.weight.data)
    #nn.init.xavier_normal_(self.enc4.weight.data)
    #nn.init.xavier_normal_(self.logvar.weight.data)
    #nn.init.xavier_normal_(self.mu.weight.data)
    #nn.init.xavier_normal_(self.dec1.weight.data)
    #nn.init.xavier_normal_(self.dec2.weight.data)
    #nn.init.xavier_normal_(self.dec3.weight.data)
    #nn.init.xavier_normal_(self.dec4.weight.data)
    #nn.init.xavier_normal_(self.dec5.weight.data)
    #nn.init.xavier_normal_(self.dec6.weight.data)

    
  def encode(self,x):
    x = x.unsqueeze(1)
    #ENCODE
    x = self.enc1s(self.enc1(x))
    x = self.enc2s(self.enc2(x))
    x = x.view(-1,self.num_flat_features(x))
    x = self.enc3s(self.enc3(x))
    x = self.enc4s(self.enc4(x))
    
    logvar = self.logvar(x)
    mu     = self.mu(x)
    
    return logvar,mu
  
  def sample(self,logvar,mu):
    return mu + torch.randn_like(mu)*torch.exp(.5*logvar)
  
  def decode(self,x):
    #DECODE
    x = self.dec1s(self.dec1(x))
    x = self.dec2s(self.dec2(x))
    x = self.dec3s(self.dec3(x))
    x = x.view(-1,256,7,7)
    x = self.dec4s(self.dec4(x))
    x = self.dec5s(self.dec5(x))
    x = self.dec6s(self.dec6(x))
    return x
  
  def forward(self,x):
    logvar,mu = self.encode(x)
    z         = self.sample(logvar,mu)
    y         = self.decode(z)
    return y
  
  def num_flat_features(self,x):
    size = x.size()[1:]
    num  = 1
    for s in size:
      num *= s
    return num
  
model     = AE()
model     = model.to(device)

lr        = 1e-4

In [ ]:
model = torch.load("model.torch").to(device)

In [ ]:
def compute_kernel(x, y):
    x_size = x.size(0)
    y_size = y.size(0)
    dim = x.size(1)
    x = x.unsqueeze(1) # (x_size, 1, dim)
    y = y.unsqueeze(0) # (1, y_size, dim)
    tiled_x = x.expand(x_size, y_size, dim)
    tiled_y = y.expand(x_size, y_size, dim)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input) # (x_size, y_size)

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

In [ ]:
#@title Training model...

#def W_loss(z):
#  return torch.norm(torch.mean(z)**2 + (torch.mean(z**2) - 1)**2)

def W_loss(z):
  return compute_mmd(z,torch.randn_like(z))


lr = 1e-3
n_ep = 300
lamb = 1
train_loss_log = np.zeros(n_ep)
test_loss_log  = np.zeros(n_ep)
prior_loss_log = np.zeros(n_ep)

optimizer = optim.Adam(model.parameters(),lr=lr)
for epoch in range(n_ep):

  model.train()
  
  for batch_idx, minibatch in enumerate(train_loader):
    minibatch = minibatch[0].to(device).squeeze(1)
    
    optimizer.zero_grad()
    
    logvar,mu = model.encode(minibatch)
    z         = model.sample(logvar,mu)
    
    output    = model.decode(z)
    
    rec_loss = F.binary_cross_entropy(output,minibatch.unsqueeze(1))
    prior_loss = lamb*W_loss(z)
    
    loss     = rec_loss + prior_loss
      
    train_loss_log[epoch] += loss.item()
    prior_loss_log[epoch] += prior_loss.item()
    
    loss.backward()
    optimizer.step()
    
  model.eval()
  with torch.no_grad():
    for batch_idx,minibatch in enumerate(test_loader):
      minibatch = minibatch[0].to(device).squeeze(1)
      
      logvar,mu = model.encode(minibatch)
      z      = model.sample(logvar,mu)
      output = model.decode(z)
      
      rec_loss = F.binary_cross_entropy(output,minibatch.unsqueeze(1))
      prior_loss = lamb*W_loss(z)
      
      loss     = rec_loss + prior_loss
      
      test_loss_log[epoch] += loss.item()
      
  if test_loss_log[epoch] < test_loss_log[epoch-1]:
    torch.save(model,'model.torch')
  
  plt.clf()
  plt.subplot(121)
  plt.plot(train_loss_log[:epoch+1]/train_loss_log[0])
  plt.plot(test_loss_log[:epoch+1]/test_loss_log[0])
  plt.plot(prior_loss_log[:epoch+1]/train_loss_log[:epoch+1])
  plt.legend(["train","test","prior"])
  plt.title("EPOCH #{}".format(epoch))
  plt.subplot(122)
  plt.imshow(output[np.random.randint(0,16)].squeeze(0).cpu().numpy())
  plt.grid()
  plt.gcf().set_size_inches(15, 10)
  display.clear_output(wait=True)
  display.display(plt.gcf())
  
   
  if (epoch+1)%5==0:
    #print("EPOCH #{} DONE, TRAIN_ERROR {}, TEST_ERROR {}".format(epoch,\
    #                                    train_loss_log[epoch],\
    #                                    test_loss_log[epoch]))
    lr /= 2.
    optimizer = optim.Adam(model.parameters(),lr=lr)
    

In [ ]:
#@title Reconstruction test
model.eval()
for i in range(3):
  with torch.no_grad():
    inp = testset[np.random.randint(0,1000)][0].to(device)
    out = model(inp).squeeze(0).squeeze(0)
    plt.subplot(121)
    plt.imshow(inp.squeeze(0).squeeze(0).cpu().numpy(),interpolation="none")
    plt.grid()
    plt.title("original")
    plt.subplot(122)
    plt.imshow(out.cpu().numpy(),interpolation="none")
    plt.grid()
    plt.title("reconstructed")
    plt.show()

In [ ]:
#@title Displaying 3D PCA latent space

def load_test_set(n):
  y = torch.cat([testset[i][0] for i in range(n)],0)
  z = [testset[i][1] for i in range(n)]
  return y,z

n = 10000

with torch.no_grad():
  x,label   = load_test_set(n)
  logvar,mu = model.encode(x.to(device))
  z         = model.sample(logvar,mu).cpu().numpy()

Rxx = np.cov(z.T)
[U,S,V] = np.linalg.svd(Rxx,full_matrices=True)
W = U[:,:3]

latent = z.dot(W)

sort = np.argsort(label)
label = np.asarray(label)[sort]
latent = latent[sort,:]

data = []

for i in range(10):
  data.append(go.Scatter3d(x=latent[label==i,0],
                     y =latent[label==i,1],
                     z =latent[label==i,2],
                     mode = 'markers',
                     marker = dict(size=2,
                                  color=i,
                                  colorscale="Jet",
                                  opacity=.8)))

fig = go.Figure(data=data)
plot(fig)

#for i in range(10):
#  plt.plot(latent[label==i,0],latent[label==i,1],"C{}.".format(i))
#plt.legend(range(10))
#plt.title("PC 1")
#plt.gcf().set_size_inches(8,8)
#plt.show()

#for i in range(10):
#  plt.plot(latent[label==i,2],latent[label==i,3],"C{}.".format(i))  
#plt.legend(range(10))
#plt.title("PC 2")
#plt.gcf().set_size_inches(8,8)
#plt.show()

In [ ]:
#@title Reconstruction from the 2D latent space
model.eval()
with torch.no_grad():
  #test = torch.tensor(np.asarray([-5,0]).dot(W.T),dtype=torch.float).to(device)
  test = torch.tensor(latent[3,:].dot(W.T),dtype=torch.float).to(device)
  img  = model.decode(test.unsqueeze(0)).squeeze(0).squeeze(0)
  plt.imshow(img.cpu().numpy())
  plt.grid()
  plt.show()

In [ ]:
#@title Animation
!rm -r /tmp/output
!mkdir /tmp/output
x = np.linspace(0,2*np.pi,200)
for i in range(200):
  with torch.no_grad():
    test = torch.tensor(np.asarray([5*np.cos(x[i]),5*np.sin(x[i])]).dot(W.T),dtype=torch.float).to(device)
    #test = torch.tensor(latent[3,:].dot(W.T),dtype=torch.float).to(device)
    img  = model.decode(test.unsqueeze(0)).squeeze(0).squeeze(0)
    plt.imshow(img.cpu().numpy(),interpolation="none")
    plt.grid()
    #plt.savefig("/tmp/output/img_{:03d}.jpg".format(i))
    plt.show()
!cd /tmp/ && zip image.zip /tmp/output/*.jpg
from google.colab import files

In [ ]:
with torch.no_grad():
  z = model.sample(*model.encode(minibatch)).cpu().numpy()
  
print("Minibatch var  = %f" % np.var(z))
print("Minibatch mean = %f" % np.mean(z))

In [ ]:
files.download('/tmp/image.zip')

In [ ]:
#@title Interpolation
y,img = load_test_set(2)
n = 20
x = torch.linspace(0,1,n).to(device)
z = model.sample(*model.encode(y.to(device)))


model.eval()
with torch.no_grad():
  for i in range(n):
    z_i = (z[0]*x[i] + z[1]*(1-x[i])).unsqueeze(0)
    img = model.decode(z_i).cpu().numpy().reshape([28,28])
    plt.imshow(img)
    plt.grid()
    plt.savefig("{:03d}.png".format(i))
    plt.show()

In [ ]:
img.shape

In [ ]:
!zip image.zip *.png